In [53]:
from openai import OpenAI
import pandas as pd
import langchain
from langchain_community.document_loaders import JSONLoader
import tiktoken
import faiss-gpu 
import langchain_experimental
import "langchain[docarray]"

SyntaxError: invalid syntax (4234274321.py, line 6)

In [43]:
politifact_filepath = '/home/envoy82/datasets/politifact_factcheck_data.json'
politifact = pd.read_json(politifact_filepath, lines=True)

In [44]:
politifact_false_or_mostly_false = politifact[(politifact['verdict'] == "false") | (politifact['verdict'] == "mostly-false")]

In [45]:
politifact_false_or_mostly_false

,verdict,statement_originator,statement,statement_date,statement_source,factchecker,factcheck_date,factcheck_analysis_link
1,false,Matt Gaetz,"""Bennie Thompson actively cheer-led riots in t...",6/7/2022,television,Yacob Reyes,6/13/2022,https://www.politifact.com/factchecks/2022/jun...
3,false,Bloggers,"""BUSTED: CDC Inflated COVID Numbers, Accused o...",2/1/2021,blog,Madison Czopek,2/5/2021,https://www.politifact.com/factchecks/2021/feb...
6,false,Michele Bachmann,"""My husband and I have never gotten a penny of...",6/26/2011,news,Robert Farley,7/1/2011,https://www.politifact.com/factchecks/2011/jul...
11,false,Carlos Beruff,"Says President Obama’s deal ""allows Iran to pr...",5/5/2016,news,Joshua Gillin,5/11/2016,https://www.politifact.com/factchecks/2016/may...
13,false,Sidney Powell,"""At least 450,000 ballots in the key states .....",11/8/2020,news,Samantha Putterman,11/20/2020,https://www.politifact.com/factchecks/2020/nov...
...,...,...,...,...,...,...,...,...
21147,mostly-false,Donald Trump,Says the large trade deficit with Japan stems ...,8/13/2019,speech,Jon Greenberg,8/15/2019,https://www.politifact.com/factchecks/2019/aug...
21148,false,Donald Trump Jr.,"""Tens of thousands"" of people leave New York e...",11/1/2019,social_media,Jill Terreri Ramos,11/8/2019,https://www.politifact.com/factchecks/2019/nov...
21149,mostly-false,Chris Abele,"""I have fought for our shared values without b...",1/4/2011,news,Dave Umhoefer,1/13/2011,https://www.politifact.com/factchecks/2011/jan...
21150,false,Bloggers,"""Germany halts all Covid-19 vaccines, says the...",8/27/2021,blog,Ciara O'Rourke,9/9/2021,https://www.politifact.com/factchecks/2021/sep...


In [46]:
politifact_false_or_mostly_false.to_json('politifact_false_or_mostly_false.json')

In [50]:
loader = JSONLoader(
    file_path='politifact_false_or_mostly_false.json',
    jq_schema='.statement',
    text_content=False,
    json_lines=False)

data = loader.load()

In [51]:
data

[Document(page_content='{"1": "\\"Bennie Thompson actively cheer-led riots in the \\u201990s.\\"", "3": "\\"BUSTED: CDC Inflated COVID Numbers, Accused of Violating Federal Law\\"", "6": "\\"My husband and I have never gotten a penny of money from the farm.\\"", "11": "Says President Obama\\u2019s deal \\"allows Iran to produce a nuclear weapon.\\"", "13": "\\"At least 450,000 ballots in the key states ... miraculously only have a mark for Joe Biden on them, and no other candidate.\\"", "16": "Tampa Bay is\\u00a0\\"leading the state in jobs creation.\\"", "17": "Says Raphael Warnock has \\"said we need to open up the prisons\\" and \\"eliminate cash bail.\\"", "19": "\\"Newly discovered video shows late night deliveries of tens of thousands of illegal ballots 8 hours after deadline.\\"", "22": "With voting by mail, \\"you get thousands and thousands of people sitting in somebody\'s living room, signing ballots all over the place.\\"", "23": "During his governorship, Virginia made \\"th